# Interview Task for adidas

## Topic Clustering using NLP on Customer Store reviews

### Loading and preprocessing data

In [2]:
# For each file in the dataset folder that is a .csv, load it in a dataframe and save it in a dictionary
# Each key is the name of the file without the extension

import os
import pandas as pd

def load_dataset(folder_name = 'dataset/adidas_input'):
    datasets = {}
    for file in os.listdir(folder_name):
        if file.endswith('.csv'):
            name = file.split('.')[0]
            datasets[name] = pd.read_csv(folder_name + "/" + file)
    return datasets



input_datasets = load_dataset()

In [3]:
list(input_datasets.keys())



['Berlin Store Originals 1',
 'Berlin Store Originals 2',
 'Frankfurt Store',
 'Herzo Outlet',
 'Herzo Store',
 'Munich Outlet',
 'Munich Terrex Store',
 'Nuremberg Store']

In [4]:
# Load dataset/review_metadata.csv
review_metadata = pd.read_csv('dataset/review_metadata.csv')
review_metadata

,store,num_ratings,num_reviews,star_rating
0,Herzo Store,8776,NaN,4.2
1,Herzo Outlet,2365,NaN,4.2
2,Nuremberg Store,763,NaN,4.3
3,Munich Outlet,450,NaN,4.3
4,Munich Terrex Store,102,NaN,4.7
5,Frankfurt Store,786,NaN,4.1
6,Berlin Store Originals,1293,NaN,4.1
7,Competitor Outlet,6533,NaN,4.4


In [5]:
# Go over each dataframe and name the columns with the following names:
# ['link', 'customer_name', 'cutomer_details', 'date_reference', 'text']

for key in input_datasets.keys():
    df = input_datasets[key]
    df.columns = ['link', 'customer_name', 'cutomer_details', 'date_reference', 'text']
    input_datasets[key] = df


In [6]:
# Take the two Berlin datasets and concatenate them
df_berlin = pd.concat([input_datasets['Berlin Store Originals 1'], input_datasets['Berlin Store Originals 2']])


# Make a new processed dictionary with the updated datasets
processed_datasets = input_datasets.copy()
processed_datasets['Berlin Store Originals'] = df_berlin

# Delete the old Berlin datasets
del processed_datasets['Berlin Store Originals 1']
del processed_datasets['Berlin Store Originals 2']

processed_datasets.keys()

dict_keys(['Frankfurt Store', 'Herzo Outlet', 'Herzo Store', 'Munich Outlet', 'Munich Terrex Store', 'Nuremberg Store', 'Berlin Store Originals'])

In [7]:
# Create a unique id and a customer id which maps to customer name for each processed dataset
# Drop the link column and the customer_name column after creating the customer_id
for key in processed_datasets.keys():
    df = processed_datasets[key]
    df['unique_id'] = df.index
    df['customer_id'] = df['customer_name'].astype('category').cat.codes
    df = df.drop(columns=['link', 'customer_name'])
    processed_datasets[key] = df




In [8]:
# Check the first 5 rows of the Herzog Store dataset
processed_datasets['Herzo Store'].head()

,cutomer_details,date_reference,text,unique_id,customer_id
0,Local Guide · 307 reviews · 875 photos,a month ago,This pair of Adidas D-Rose was a blast of a sh...,0,1375
1,Local Guide · 69 reviews · 45 photos,7 months ago,Just Amazing. You can find very high quality g...,1,2112
2,Local Guide · 72 reviews · 61 photos,9 months ago,easily accessible through 200 bus from bahnhof...,2,2057
3,Local Guide · 159 reviews · 311 photos,a year ago,What an extravagant location new fixtures almo...,3,1298
4,Local Guide · 10 reviews · 15 photos,a year ago,Honestly this is heartbreaking for me because ...,4,1402


In [9]:
# Create local_guide boolean column
# If the cutomer_details contains 'Local Guide' then set it to True, else False
for key in processed_datasets.keys():
    df = processed_datasets[key]
    df['local_guide'] = df['cutomer_details'].str.contains('Local Guide')
    processed_datasets[key] = df

In [10]:
# Check how many local guides are in the Herzo Store dataset
processed_datasets['Herzo Store']['local_guide'].value_counts()

local_guide
True     1618
False     583
Name: count, dtype: int64

In [11]:
# Create date column from the date_reference
# The date_reference is in the format "a month ago, 2 months ago, 3 months ago, etc."
# or "a year ago, 2 years ago, 3 years ago, etc."
# or "a day ago, 2 days ago, 3 days ago, etc."
# or "a week ago, 2 weeks ago, 3 weeks ago, etc."
# I want to convert this to a date in the format YYYY

# If it is a day, week or "a month" or up to 3 months ago, I will use the current year
# If it is more than 3 months ago, I will use the previous year
# If it is a year or more ago, I will use current year minus the number of years mentioned

from datetime import datetime
import numpy as np

def convert_date(date_reference):
    if 'day' in date_reference:
        return datetime.now().year
    if 'week' in date_reference:
        return datetime.now().year
    if 'month' in date_reference:
        if 'a month' in date_reference:
            return datetime.now().year
        if '2 months' in date_reference:
            return datetime.now().year
        if '3 months' in date_reference:
            return datetime.now().year
        else:
            return datetime.now().year - 1
    if 'year' in date_reference:
        if 'a year' in date_reference:
            return datetime.now().year - 1
        else:
            return datetime.now().year - int(date_reference.split()[0])
    return np.nan

for key in processed_datasets.keys():
    df = processed_datasets[key]
    df['date_year'] = df['date_reference'].apply(convert_date).astype('Int64')
    df = df.drop(columns=['date_reference'])
    processed_datasets[key] = df



In [12]:
# Check what is the distribution of years for one dataset
processed_datasets['Herzo Store'].date_year.value_counts()

date_year
2020    516
2019    417
2023    404
2018    295
2022    261
2021    204
2017     56
2024     48
Name: count, dtype: Int64

In [13]:
processed_datasets['Herzo Store']

,cutomer_details,text,unique_id,customer_id,local_guide,date_year
0,Local Guide · 307 reviews · 875 photos,This pair of Adidas D-Rose was a blast of a sh...,0,1375,True,2024
1,Local Guide · 69 reviews · 45 photos,Just Amazing. You can find very high quality g...,1,2112,True,2023
2,Local Guide · 72 reviews · 61 photos,easily accessible through 200 bus from bahnhof...,2,2057,True,2023
3,Local Guide · 159 reviews · 311 photos,What an extravagant location new fixtures almo...,3,1298,True,2023
4,Local Guide · 10 reviews · 15 photos,Honestly this is heartbreaking for me because ...,4,1402,True,2023
...,...,...,...,...,...,...
2196,Local Guide · 212 reviews · 181 photos,Great offers and competent staff,2196,1769,True,2019
2197,1 review,A queue of at least 200 people !!!!!!,2197,1406,False,2022
2198,2 reviews,Great prices and service!!!,2198,1440,False,2021
2199,56 reviews,"Terrible. It's like Norma. Stick with it, stuff",2199,462,False,2020


In [14]:
# Reorder the columns to be ['unique_id', 'customer_id', 'local_guide', 'date_year', 'text']
# Save the processed datasets in a new folder called dataset/processed_dataset
# Check if the folder exists, if not create it

if not os.path.exists('dataset/processed_dataset'):
    os.makedirs('dataset/processed_dataset')

for key in processed_datasets.keys():
    df = processed_datasets[key]
    df = df[['unique_id', 'customer_id', 'local_guide', 'date_year', 'text']]
    df.to_csv('dataset/processed_dataset/' + key + '.csv', index=False)

In [15]:
# Do the same processing as above for dataset/competitor_input

# Load the competitor datasets
competitor_datasets = load_dataset('dataset/competitor_input')

# Iterate over each key in the competitor_datasets dictionary
for key in competitor_datasets.keys():
    # Rename the columns of the dataframe
    df = competitor_datasets[key]
    df.columns = ['link', 'customer_name', 'cutomer_details', 'date_reference', 'text', 'text_response']
    competitor_datasets[key] = df

    # Create unique_id and customer_id columns
    df = competitor_datasets[key]
    df['unique_id'] = df.index
    df['customer_id'] = df['customer_name'].astype('category').cat.codes
    df = df.drop(columns=['link', 'customer_name'])
    competitor_datasets[key] = df

    # Create local_guide column
    df = competitor_datasets[key]
    df['local_guide'] = df['cutomer_details'].str.contains('Local Guide')
    competitor_datasets[key] = df

    # Convert date_reference to date_year
    df = competitor_datasets[key]
    df['date_year'] = df['date_reference'].apply(convert_date).astype('Int64')
    df = df.drop(columns=['date_reference'])
    competitor_datasets[key] = df

    # Save the dataset in the processed_dataset folder
    df = df[['unique_id', 'customer_id', 'local_guide', 'date_year', 'text', 'text_response']]
    df.to_csv('dataset/processed_dataset/' + key + '.csv', index=False)


# Print the distribution of years for the "Competitor Outlet" dataset
competitor_datasets["Competitor Outlet"]['date_year'].value_counts()

date_year
2023    567
2020    401
2019    306
2018    223
2022    216
2021    169
2024     58
2017     55
2016      8
2015      3
2013      1
Name: count, dtype: Int64

In [16]:
# Merge all processed datasets from the folder dataset/processed_dataset into a single dataframe
# Save this dataframe as dataset/processed_dataset/all_datasets.csv
# Add a new column called dataset_name which contains the name of the dataset
# Add a new column called dataset_type which contains the type of the dataset (adidas or competitor)

all_datasets = pd.DataFrame()

# Check if folder combined_dataset exists, if not create it
if not os.path.exists('dataset/combined_dataset'):
    os.makedirs('dataset/combined_dataset')

for file in os.listdir('dataset/processed_dataset'):
    current_df = pd.read_csv('dataset/processed_dataset/' + file)
    current_df['store_name'] = file.split('.')[0]
    if 'Competitor' in file:
        current_df['company_name'] = 'competitor'
    else:
        current_df['company_name'] = 'adidas'

    # Convert date_year to integer
    current_df['date_year'] = current_df['date_year'].astype('Int64')


    all_datasets = pd.concat([all_datasets, current_df])



In [17]:
# Check for nans in all columns
all_datasets.isnull().sum()


unique_id           0
customer_id         0
local_guide         1
date_year           2
text               25
store_name          0
company_name        0
text_response    6769
dtype: int64

In [18]:
# Drop rows with nans in the local_guide, date_year and text columns
all_datasets = all_datasets.dropna(subset=['local_guide', 'date_year', 'text'])

# Convert date_year to integer
all_datasets['date_year'] = all_datasets['date_year'].astype('Int64')

# Check for nans in all columns
all_datasets.isnull().sum()


unique_id           0
customer_id         0
local_guide         0
date_year           0
text                0
store_name          0
company_name        0
text_response    6741
dtype: int64

In [19]:
# Save the dataset to csv
all_datasets.to_csv('dataset/combined_dataset/all_datasets.csv', index=False)


## Review metadata table

In [20]:
# Add the number of reviews for each dataset in the review_metadata dataframe
# The store column should be the same as the key in the processed_datasets dictionary 
# Map the number of reviews to the store in the review_metadata dataframe
for file in os.listdir('dataset/processed_dataset'):
    current_df = pd.read_csv('dataset/processed_dataset/' + file)
    store = file.split('.')[0]
    review_metadata.loc[review_metadata['store'] == store, 'num_reviews'] = current_df.shape[0]
    review_metadata["num_reviews"] = review_metadata["num_reviews"].astype('Int64')

review_metadata


,store,num_ratings,num_reviews,star_rating
0,Herzo Store,8776,2201,4.2
1,Herzo Outlet,2365,1091,4.2
2,Nuremberg Store,763,288,4.3
3,Munich Outlet,450,210,4.3
4,Munich Terrex Store,102,68,4.7
5,Frankfurt Store,786,324,4.1
6,Berlin Store Originals,1293,681,4.1
7,Competitor Outlet,6533,2008,4.4


In [21]:
# save the review_metadata dataframe to dataset/review_metadata.csv
review_metadata.to_csv('dataset/review_metadata.csv', index=False)